In [1]:
from __future__ import division
import cvxpy as cx
import matplotlib.pyplot as plt
from scipy import optimize
%matplotlib inline

import tf_utils as tfu
from load_data import *
from features import *

In [2]:
def solve_lp_down(C, nu, phi_x, y_true):
    k = len(C)
    y_down = cx.Variable(k)
    y_up = cx.Variable(k)
    v = cx.Variable()
    u = cx.Variable()
    ones = np.ones((k,1))
    nu = nu.reshape((-1, 1))
    
    C_aug = C + ones.dot(nu.T).dot(phi_x - phi_x.dot(y_true).dot(ones.T))
    constraints_down = [
        v <= C_aug * y_down,
        cx.sum_entries(y_down) == 1,
        y_down >= 0,
    ]
    problem_down = cx.Problem(cx.Maximize(v), constraints_down)
    problem_down.solve()

    return y_down.value, v.value

def solve_lp_up(C, nu, phi_x):
    k = len(C)
    y_up = cx.Variable(k)
    u = cx.Variable()
    ones = np.ones((k,1))
    nu = nu.reshape((-1, 1))
    C_aug = C + ones.dot(nu.T).dot(phi_x)
    constraints = [
        u >= y_up.T * C_aug,
        cx.sum_entries(y_up) == 1,
        y_up >= 0
    ]
    problem = cx.Problem(cx.Minimize(u), constraints)
    problem.solve()
    return y_up.value, u.value

In [3]:
def phi_basic(X, k):
    return np.stack([
        [np.concatenate([np.outer(x,x).ravel() * (j == i) for j in range(k)])
         for x in X] for i in range(k)
        ], axis=2)

def train_basic(X, Y, phi, opts):
    (nx, dx), (ny, k) = X.shape, Y.shape
    assert nx == ny

    if opts.c_mode == 'random':                                            
        C = np.random.uniform(size=(k, k))
    elif opts.c_mode == 'ones':
        C = np.ones((k, k))
    else:
        raise ValueError('C mode: %s' % opts.mode)
    C[range(k), range(k)] = 0

    # precompute phi since it does not change                                    
    Phi_ndk = phi(X, k)
    d = Phi_ndk.shape[1]
    nu = np.zeros((d, 1))

    def func_grad(nu):
        f, g = 0, 0
        for phi_x, y_true in zip(Phi_ndk, Y[..., None]):
            # solve the inner LP                                                 
            y_down_star, v = solve_lp_down(C, nu, phi_x, y_true)

            # compute the subgradient
            f += v
            g += phi_x.dot(y_down_star - y_true)
        return f / len(Y), g / len(Y)
    
    def callback(nu):
        print func_grad(nu)[0]
    
    nu, v_star, info = optimize.fmin_l_bfgs_b(
        func_grad, nu, callback=callback, pgtol=opts.tol, maxiter=opts.iters
    )
    info.pop('grad')
    print v_star, info
    
    def classifier(X_bd):
        Y = []
        Phi_bdk = phi(X_bd, k)
        for phi_x in Phi_bdk:
            y_up_star, u = solve_lp_up(C, nu, phi_x)
            Y.append(y_up_star)
        return np.squeeze(Y)

    return tfu.struct(nu=nu, C=C, predict=classifier)

def train_ours(X, Y, phi, opts):
    (nx, dx), (ny, k) = X.shape, Y.shape
    assert nx == ny

    if opts.c_mode == 'random':                                            
        C = np.random.uniform(size=(k, k))
    elif opts.c_mode == 'ones':
        C = np.ones((k, k))
    else:
        raise ValueError('C mode: %s' % opts.mode)
    C[range(k), range(k)] = 0
    
    nu = np.zeros([opts.dim_nu, 1] )
    
    for it in range(opts.iters):
        loss, grad = [], []
        for x, y_true in zip(X, Y[..., None]):
            # solve the inner LP
            phi_x = np.stack([
                phi.phi(x=x[None], y=one_hot(i, k)[None])
                for i in range(k)
            ], axis=2)[0]
            y_down_star, v = solve_lp_down(C, nu, phi_x, y_true)

            # compute the subgradient
            loss.append(v)
            grad.append(np.outer(nu, y_down_star - y_true))
            nu -= opts.alpha_nu * phi_x.dot(y_down_star - y_true)
        loss, grad  = np.mean(loss), np.mean(grad, axis=0)
        print it, loss,  phi.train(x=X, g=grad)
        

In [4]:
X, y = load('iris')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=30)

In [8]:
opts = tfu.struct(
    iters=50,
    tol=1e-5,
    c_mode='ones',
    
    alpha_nu=5e-2,
    dim_x=X.shape[1],
    dim_y=y.shape[1],
    dim_nu=64,
    sizes=[],
    
    solver_type='Adam',
    alpha=1e-3,
    beta1=0.9,
    beta2=0.999,
)
# features = Features(opts)

In [9]:
clf = train_ours(X_train, y_train, features, opts)
# clf = train_basic(X_train, y_train, phi_basic, opts)

0 0.667106182973 {'it': 36, 'op': None}
1 0.666440860378 {'it': 37, 'op': None}
2 0.665765082539 {'it': 38, 'op': None}
3 0.66507988318 {'it': 39, 'op': None}
4 0.664386038877 {'it': 40, 'op': None}
5 0.663684089652 {'it': 41, 'op': None}
6 0.662974356852 {'it': 42, 'op': None}
7 0.662256963236 {'it': 43, 'op': None}
8 0.661531849487 {'it': 44, 'op': None}
9 0.660798792162 {'it': 45, 'op': None}
10 0.660057421857 {'it': 46, 'op': None}
11 0.659307240329 {'it': 47, 'op': None}
12 0.658547638246 {'it': 48, 'op': None}
13 0.657777912295 {'it': 49, 'op': None}
14 0.656997282131 {'it': 50, 'op': None}
15 0.656204906777 {'it': 51, 'op': None}
16 0.655399899805 {'it': 52, 'op': None}
17 0.654581345644 {'it': 53, 'op': None}
18 0.653748307854 {'it': 54, 'op': None}
19 0.652899848555 {'it': 55, 'op': None}
20 0.652035029368 {'it': 56, 'op': None}
21 0.651152928489 {'it': 57, 'op': None}
22 0.650252640382 {'it': 58, 'op': None}
23 0.64933328477 {'it': 59, 'op': None}
24 0.649041209462 {'it': 60,

In [ ]:
y_hat_train = clf.predict(X_train)
train_acc = np.equal(y_hat_train.argmax(-1), y_train.argmax(-1)).mean()
train_loss = np.einsum('bi,ij,bj->b', y_hat_train, clf.C, y_train).mean()

y_hat_test = clf.predict(X_test)
test_acc = np.equal(y_hat_test.argmax(-1), y_test.argmax(-1)).mean()
test_loss = np.einsum('bi,ij,bj->b', y_hat_test, clf.C, y_test).mean()

print 'train / test'
print 'accuracy:', train_acc, test_acc
print 'loss:', train_loss, test_loss

In [ ]:
features.phi(x=X[None, 0], y=one_hot(1, 3)[None] ).shape

In [ ]:
phi = features.phi.outputs
theta = features.params

In [ ]:
print tf.gradients(phi, theta)
print features.param_shapes